In [ ]:
!python -m spacy download en

In [ ]:
from datascience import *
import spacy
from collections import Counter
import folium

In [ ]:
data = Table.read_table('data/AHA-jobs.csv')

In [ ]:
data.show()

In [ ]:
nlp = spacy.load('en')

In [ ]:
data['job_description'][0]

In [ ]:
parsed_text = nlp(str(data['job_description'][0]))

In [ ]:
len(parsed_text)

In [ ]:
len(list(parsed_text.sents))

In [ ]:
sents_tab = Table()
sents_tab.append_column(label="Sentence", values=[sentence.text for sentence in parsed_text.sents])
sents_tab.show()

In [ ]:
toks_tab = Table()
toks_tab.append_column(label="Word", values=[word.text for word in parsed_text])
toks_tab.show()

In [ ]:
toks_tab.append_column(label="POS", values=[word.pos_ for word in parsed_text])
toks_tab.show()

In [ ]:
toks_tab.append_column(label="Lemma", values=[word.lemma_ for word in parsed_text])
toks_tab.show()

In [ ]:
def tablefy(parsed_text):
    toks_tab = Table()
    toks_tab.append_column(label="Word", values=[word.text for word in parsed_text])
    toks_tab.append_column(label="POS", values=[word.pos_ for word in parsed_text])
    toks_tab.append_column(label="Lemma", values=[word.lemma_ for word in parsed_text])
    toks_tab.append_column(label="Stop Word", values=[word.is_stop for word in parsed_text])
    toks_tab.append_column(label="Punctuation", values=[word.is_punct for word in parsed_text])
    toks_tab.append_column(label="Space", values=[word.is_space for word in parsed_text])
    toks_tab.append_column(label="Number", values=[word.like_num for word in parsed_text])
    toks_tab.append_column(label="OOV", values=[word.is_oov for word in parsed_text])
    toks_tab.append_column(label="Dependency", values=[word.dep_ for word in parsed_text])
    return toks_tab

In [ ]:
tablefy(parsed_text).show()

In [ ]:
all_descriptions = tablefy(nlp(' '.join(data['job_description'])))
all_descriptions.show(100)

In [ ]:
adjectives = all_descriptions.where('POS', are.equal_to('ADJ'))
adjectives.show(10)

In [ ]:
Counter(adjectives['Word']).most_common()

In [ ]:
adjectives = all_descriptions.where('POS', are.equal_to('ADJ')).where('Stop Word', are.equal_to(False))
Counter(adjectives['Word']).most_common()

In [ ]:
adjectives = all_descriptions.where('POS', are.equal_to('NOUN')).where('Stop Word', are.equal_to(False))
Counter(adjectives['Word']).most_common()

# Challenge
Verbs

In [ ]:
import requests
import time
import urllib

In [ ]:
latitude = []
longitude = []
for i in range(data.num_rows):
    search = urllib.parse.quote(data['employer'][i])
    
    print(data['employer'][i])

    try:
        json_res = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(search)).json()
        coordinates = json_res['results'][0]['geometry']['location']
        latitude.append(coordinates['lat'])
        longitude.append(coordinates['lng'])
    except:
        latitude.append('')
        longitude.append('')

    time.sleep(.5)

In [ ]:
data = data.with_columns('latitude', latitude, 'longitude', longitude)
data.show(5)

In [ ]:
for r in data.rows:
    print(type(r[-2]))
    break

In [ ]:
from IPython.display import HTML, display, IFrame

mapa = folium.Map(location=[39.8333333,-98.585522], zoom_start=3) # Folium is a useful library for generating
                                                                   # Google maps-like map visualizations.
for r in data.rows:
    
    if r[-2] != '':

        folium.Marker((float(r[-2]), float(r[-1]))).add_to(mapa)

mapa.save("map1.html")
IFrame('map1.html', width=700, height=400)